In [1]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
if tf.__version__.startswith("2."):
  import tensorflow.compat.v1 as tf
  tf.compat.v1.disable_v2_behavior()
  tf.compat.v1.disable_eager_execution()
  print("Enabled compatitility to tf1.x")

Instructions for updating:
non-resource variables are not supported in the long term
Enabled compatitility to tf1.x


In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
data = np.load('proyecto_training_data.npy')
column_names = ["SalePrice","OverallQual","1stFlrSF","TotRmsAbrGrd","YearBuilt","LotFrontage"]
data_df = pd.DataFrame(data, columns=column_names)

In [4]:
data_df

,SalePrice,OverallQual,1stFlrSF,TotRmsAbrGrd,YearBuilt,LotFrontage
0,208500.0,7.0,856.0,8.0,2003.0,65.0
1,181500.0,6.0,1262.0,6.0,1976.0,80.0
2,223500.0,7.0,920.0,6.0,2001.0,68.0
3,140000.0,7.0,961.0,7.0,1915.0,60.0
4,250000.0,8.0,1145.0,9.0,2000.0,84.0
...,...,...,...,...,...,...
1455,175000.0,6.0,953.0,7.0,1999.0,62.0
1456,210000.0,6.0,2073.0,7.0,1978.0,85.0
1457,266500.0,7.0,1188.0,9.0,1941.0,66.0
1458,142125.0,5.0,1078.0,5.0,1950.0,68.0


In [5]:
correlacion = np.corrcoef(data,rowvar=False)[0,:]
pd.DataFrame(correlacion[1:].reshape(1,5),columns=column_names[1:])

,OverallQual,1stFlrSF,TotRmsAbrGrd,YearBuilt,LotFrontage
0,0.790982,0.605852,0.533723,0.522897,NaN


### Variable con mayor correlacion "OverallQual"

In [8]:
dataset = data[:,0:2]
matriz_x = np.hstack((dataset[:,1].reshape(dataset[:,1].size,1),np.ones((dataset[:,1].size,1))))
data_x = data[:,1]
data_y = data[:,0]

In [82]:
np.array(data_x).reshape(-1,1)

array([[7.],
       [6.],
       [7.],
       ...,
       [7.],
       [5.],
       [5.]])

In [77]:
def error(y, y_pred):
    return 1/2*tf.reduce_mean(tf.math.square(y - y_pred))

In [98]:
lr = 0.01
batch_size = 5
epochs = 5
total_iteraciones = int(data_x.size/batch_size)

tf.reset_default_graph()
g = tf.Graph()
with g.as_default():
    parametros_entrenables = tf.get_variable(name="Parametros_Entrenables", dtype=tf.float32, #shape=[2,1], 
                                initializer=[[0.5],[1.0]])
    
    tensor_x = tf.placeholder(dtype=tf.float32,name="X")
    tensor_y = tf.placeholder(dtype=tf.float32,name="Y")

    with tf.name_scope("Matriz_X"):
        matriz_x = tf.stack([tensor_x, tf.ones(tf.shape(tensor_x))],1)
    with tf.name_scope("Model"):
        y_pred = tf.linalg.matmul(matriz_x,parametros_entrenables)
    with tf.name_scope("MSE"):
        MSE = error(tensor_y, y_pred)
        
    with tf.name_scope("Gradient_Descent"):
        gradientes = tf.gradients(MSE,parametros_entrenables, name="Gradients")
        actualizacion_parametros = tf.assign(parametros_entrenables,parametros_entrenables - lr*gradientes[0], name="Parameters_Update")
    with tf.train.MonitoredSession() as session:
        for epoch in range(epochs):
            for i in range(total_iteraciones):
                muestra_inicio = i * batch_size
                muestra_fin = muestra_inicio + batch_size
                x = np.array(data_x[muestra_inicio:muestra_fin])
                y = np.array(data_y[muestra_inicio:muestra_fin])
                feed_dict = {tensor_x:x, tensor_y:y}
                parametros = session.run([actualizacion_parametros],feed_dict=feed_dict)
                print("Epoch {} iteracion {} m={} b={}".format(epoch,i,parametros[0][0],parametros[0][1]))
            feed_dict = {tensor_x:data_x, tensor_y:data_y}
            predicciones = session.run(y_pred,feed_dict=feed_dict)
        



teracion 176 m=[19716.658] b=[51844.71]
Epoch 3 iteracion 177 m=[20740.89] b=[52078.766]
Epoch 3 iteracion 178 m=[19755.402] b=[51905.484]
Epoch 3 iteracion 179 m=[22693.283] b=[52512.836]
Epoch 3 iteracion 180 m=[21364.367] b=[52333.883]
Epoch 3 iteracion 181 m=[21247.848] b=[52361.953]
Epoch 3 iteracion 182 m=[20072.756] b=[52142.51]
Epoch 3 iteracion 183 m=[18886.855] b=[52017.363]
Epoch 3 iteracion 184 m=[20127.814] b=[52236.53]
Epoch 3 iteracion 185 m=[21690.094] b=[52494.22]
Epoch 3 iteracion 186 m=[21699.82] b=[52548.59]
Epoch 3 iteracion 187 m=[21961.574] b=[52638.312]
Epoch 3 iteracion 188 m=[20806.76] b=[52472.883]
Epoch 3 iteracion 189 m=[20778.31] b=[52513.934]
Epoch 3 iteracion 190 m=[19491.805] b=[52272.92]
Epoch 3 iteracion 191 m=[17837.48] b=[52023.7]
Epoch 3 iteracion 192 m=[19471.38] b=[52320.99]
Epoch 3 iteracion 193 m=[18271.83] b=[52127.81]
Epoch 3 iteracion 194 m=[17154.518] b=[51987.676]
Epoch 3 iteracion 195 m=[16582.803] b=[51940.254]
Epoch 3 iteracion 196 m=[1